In [ ]:
!pip install flask
!pip install pyngrok

In [ ]:
import os

# Create a directory named "templates" in the current working directory
os.makedirs("templates", exist_ok=True)

In [ ]:
from pyngrok import ngrok
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk, re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Set authentication token
ngrok.set_auth_token("2c54S0BCzBB0ZiGw5FSS82ulUNb_7wsNuyyPb22woziTCnnXn")

In [ ]:
import joblib
import threading
import time

# Load the transformed_text function
def text_preprocessing(text):
  #lower casing
  text = text.lower()

  #remove punctuations
  text = re.sub(r'[^\w\s]', '', text)

  #remove stopwords
  new_text = []
  for word in text.split():
    if word in stopwords.words('english'):
      new_text.append('')
    else:
      new_text.append(word)
  x=new_text[:]
  new_text.clear
  text =  ' '.join(x)

  #Tokenize
  text = word_tokenize(text)

  #Stemming
  new_list = []
  ps = PorterStemmer()
  for word in text:
    new_list.append(ps.stem(word))
  text = " ".join(new_list)

  return text

# Load the TF-IDF vectorizer
tfidf_vectorizer = joblib.load('tfidf.pkl')  # Replace 'tfidf_vectorizer.pkl' with the path to your TF-IDF vectorizer pickle file

# Load the model
b_model = joblib.load('bmodel.pkl')  # Replace 'spam_classifier_model.pkl' with the path to your trained model pickle file


#Start flask app
def start_flask_app():
  from flask import Flask,request,jsonify,render_template

  app=Flask(__name__)

  @app.route('/')
  def index():
    return render_template('index.html')

  @app.route('/analyze',methods=['POST'])
  def analyze_review():
    review = request.json['review']
    sentiment = analyze_sentiment(review)
    print(sentiment)  # Log the sentiment to the output cell
    #sentiment = "Negative"
    return jsonify({'sentiment': sentiment})

  def analyze_sentiment(review):
    preprocessed_text = text_preprocessing(review)
    vectorize_text = tfidf_vectorizer.transform([preprocessed_text])
    prediction = b_model.predict(vectorize_text.toarray())
    if prediction[0] == 0:
      return "Negative"
    elif prediction[0] == 1:
      return "Somewhat Negative"
    elif prediction[0] == 2:
      return "Neutral"
    elif prediction[0] == 3:
      return "Somewhat Positive"
    elif prediction[0] == 4:
      return "Positive"

  app.run(port=5011)

flask_thread = threading.Thread(target=start_flask_app)
flask_thread.start()

# Wait for the Flask app to start
time.sleep(2)

# Start ngrok tunnel
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL
print(' * Tunnel URL:', ngrok_tunnel.public_url)

# Keep the Colab session alive
while True:
    time.sleep(3600)  # Keep the session alive for 1 hour



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5011
INFO:werkzeug:Press CTRL+C to quit


 * Tunnel URL: https://08a0-35-229-78-5.ngrok-free.app


INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 07:01:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 07:01:29] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 07:01:31] "POST /analyze HTTP/1.1" 200 -


Somewhat Positive


...

INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 07:08:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 07:08:56] "POST /analyze HTTP/1.1" 200 -


Somewhat Positive


INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 07:09:20] "POST /analyze HTTP/1.1" 200 -


Somewhat Negative


KeyboardInterrupt: 